In [ ]:
! pip install xgboost category_encoders daal4py

In [1]:
import numpy as np 
import pandas as pd 
import xgboost as xgb
import daal4py as d4p
import time 

/usr/local/lib/python3.6/dist-packages/pkg_resources/__init__.py:119: PkgResourcesDeprecationWarning: 0.18ubuntu0.18.04.1 is an invalid version and will not be supported in a future release
  PkgResourcesDeprecationWarning,


In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
meal= pd.read_csv('meal_info.csv')
ful = pd.read_csv('fulfilment_center_info.csv')
sub = pd.read_csv('sample_submission.csv')
print('Done.')

Done.


In [4]:
print('train shape',train.shape)
print('test shape',test.shape)
print('meal shape',meal.shape)
print('fulfillment shape',ful.shape)

train shape (456548, 9)
test shape (32573, 8)
meal shape (51, 3)
fulfillment shape (77, 5)


In [5]:
# combine both train and test, then merge ful and meal to the data .
ntrain = train.shape[0]
ntest = test.shape[0]
all_data = pd.concat((train,test)).reset_index(drop=True)
print(f'The shape of the combined dataframe is: {all_data.shape}')

The shape of the combined dataframe is: (489121, 9)


In [6]:
data = pd.merge(all_data, meal, on="meal_id", how="outer")
data = pd.merge(data, ful, on="center_id", how="outer")

In [7]:
train = data[:ntrain].copy()
test = data[ntrain:].copy()

In [8]:
train.shape,test.shape

((456548, 15), (32573, 15))

In [9]:
train.drop(columns =['center_id','meal_id'],inplace=True)
test.drop(columns =['center_id','meal_id'],inplace=True)

In [10]:
from sklearn.compose import make_column_selector as selector
categorical_columns_selector = selector(dtype_include=object)
categorical_columns = categorical_columns_selector(train)
categorical_columns

['category', 'cuisine', 'center_type']

In [11]:
# combine both train and test, then merge ful and meal to the data .
ntrain = train.shape[0]
ntest = test.shape[0]
all_data = pd.concat((train,test)).reset_index(drop=True)
print(f'The shape of the combined dataframe is: {all_data.shape}')

The shape of the combined dataframe is: (489121, 13)


In [12]:
import category_encoders as ce
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(all_data, all_data['num_orders'], test_size=0.3,
                                                    random_state=0)
X_train.shape, X_test.shape

ord_enc = ce.OrdinalEncoder(cols=['category', 'cuisine', 'center_type']).fit(X_train,y_train)


data = ord_enc.transform(all_data)
data.head()

main_cols =data.columns.difference(['num_orders'])

In [13]:
train = data[:ntrain].copy()
#train.drop_duplicates(inplace = True, ignore_index=True)
target = train.num_orders.copy()
train.drop(['num_orders','id'] ,axis=1, inplace=True)

test = data[ntrain:].copy()
test.drop(['num_orders','id'] ,axis=1, inplace=True)
test = test.reset_index(drop=True)

In [14]:
target = target.fillna(target.mean())

In [51]:
train = train.rename(columns={'week':1, 'checkout_price':2, 'base_price':3, 'emailer_for_promotion':4,
       'homepage_featured':5, 'category':6, 'cuisine':7, 'city_code':8, 'region_code':9,
       'center_type':10, 'op_area':11})

In [52]:
train.columns

Int64Index([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11], dtype='int64')

In [53]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(train, target, test_size = 0.30)

X_train.shape,y_train.shape

((319583, 11), (319583,))

In [54]:
clf = xgb.XGBClassifier(n_estimators=1000, max_depth=7, eta=0.1, subsample=0.7, colsample_bytree=0.8)
reg = xgb.XGBRegressor(n_estimators=1000, max_depth=7, eta=0.1, subsample=0.7, colsample_bytree=0.8)

results = pd.DataFrame(columns=["Regressor", "Train Time", "Stock Inference Time", "Daal Inference Time"])

In [57]:
start = time.time()
xgb_model = reg.fit(X_train, y_train)
train_time = time.time()-start


daal_model = d4p.get_gbt_model_from_xgboost(xgb_model.get_booster())
daal_start = time.time()
d4p.gbt_regression_prediction().compute(X_val, daal_model)
daal_time = time.time()-daal_start

pred_start = time.time()
pred = xgb_model.predict(X_val)
pred_time = time.time()-pred_start

df2 = pd.DataFrame({"Regressor": "XGBoost","Train Time": train_time,"Stock Inference Time": pred_time, "Daal Inference Time" : daal_time}, index = [0])
results = results.append(df2, ignore_index = True)
results

,Regressor,Train Time,Stock Inference Time,Daal Inference Time
0,XGBoost,59.84019,0.465919,0.253114
